# Proportional Score Voting

Also called Reweighted Range Voting

In [88]:
import pandas

ballots = [
  {"Red": 5, "Green": 0, "Yellow": 3, "Blue": 5},
  {"Red": 5, "Green": 0, "Yellow": 0, "Blue": 4},
  {"Red": 0, "Green": 5, "Yellow": 0, "Blue": 1},
  {"Red": 1, "Green": 2, "Yellow": 4, "Blue": 3}, 
  {"Red": 1, "Green": 0, "Yellow": 2, "Blue": 0},  
  {"Red": 1, "Green": 3, "Yellow": 0, "Blue": 1},
  {"Red": 0, "Green": 0, "Yellow": 5, "Blue": 0},
  {"Red": 5, "Green": 0, "Yellow": 0, "Blue": 4},
]

seats = 4
seated = []
max_score = max(max(ballot.values()) for ballot in ballots)

def reweight(ballot):
  seated_scores = [
      ballot[candidate] for candidate in ballot if candidate in seated
  ]
  weight = 1/(1+sum(seated_scores)/max_score)
  return {candidate: weight*ballot[candidate] for candidate in ballot}

def nextRound(ballots):
  reweightedBallots = [reweight(ballot) for ballot in ballots] 
  winner = pandas.DataFrame(reweightedBallots).sum().drop(seated).idxmax()
  print(pandas.DataFrame(reweightedBallots).sum())
  seated.append(winner)
  return reweightedBallots

while len(seated) < seats:
  nextRound(ballots)
  print(seated)


Red       18.0
Green     10.0
Yellow    14.0
Blue      18.0
dtype: float64
['Red']
Red       10.000000
Green      9.166667
Yellow    11.500000
Blue      10.833333
dtype: float64
['Red', 'Yellow']
Red       8.881410
Green     8.500000
Yellow    6.903846
Blue      9.256410
dtype: float64
['Red', 'Yellow', 'Blue']
Red       6.684219
Green     7.078755
Yellow    6.121795
Blue      6.947497
dtype: float64
['Red', 'Yellow', 'Blue', 'Green']


# Sequential Proportional Approval Voting

This is functionally equivalent to Proportional Score Voting, just max_score is 1.

In [89]:
import pandas

ballots = [
    {"Red": 1, "Green": 0, "Yellow": 0, "Blue": 1},
    {"Red": 0, "Green": 1, "Yellow": 0, "Blue": 1},
    {"Red": 1, "Green": 0, "Yellow": 1, "Blue": 0},
    {"Red": 1, "Green": 0, "Yellow": 1, "Blue": 1},
    {"Red": 0, "Green": 1, "Yellow": 0, "Blue": 1},
    {"Red": 1, "Green": 0, "Yellow": 1, "Blue": 1},
    {"Red": 1, "Green": 1, "Yellow": 0, "Blue": 1},
    {"Red": 0, "Green": 1, "Yellow": 0, "Blue": 1},
    {"Red": 1, "Green": 0, "Yellow": 0, "Blue": 1},
]

SEATS = 4

seated = []
max_score = max(max(ballot.values())
                for ballot in ballots)  # 1 for Approval Voting


def reweight(ballot):
    seated_scores = [
        ballot[candidate] for candidate in ballot if candidate in seated
    ]
    weight = 1/(1+sum(seated_scores)/max_score)
    return {candidate: weight*ballot[candidate] for candidate in ballot}


def nextRound(ballots):
    reweighted_ballots = [reweight(ballot) for ballot in ballots]
    winner = pandas.DataFrame(reweighted_ballots).sum().drop(seated).idxmax()
    print(pandas.DataFrame(reweighted_ballots).sum())
    seated.append(winner)
    return reweighted_ballots


while len(seated) < SEATS:
    nextRound(ballots)
    print(f"Winners: {seated}")


Red       6.0
Green     4.0
Yellow    3.0
Blue      8.0
dtype: float64
Winners: ['Blue']
Red       3.5
Green     2.0
Yellow    2.0
Blue      4.0
dtype: float64
Winners: ['Blue', 'Red']
Red       2.166667
Green     1.833333
Yellow    1.166667
Blue      3.166667
dtype: float64
Winners: ['Blue', 'Red', 'Green']
Red       2.083333
Green     1.250000
Yellow    1.166667
Blue      2.583333
dtype: float64
Winners: ['Blue', 'Red', 'Green', 'Yellow']


In [90]:
import pandas

ballots = [
  {"Red": 1, "Green": 0, "Yellow": 0, "Blue": 1},
  {"Red": 0, "Green": 1, "Yellow": 0, "Blue": 1},
  {"Red": 1, "Green": 0, "Yellow": 1, "Blue": 0}, 
  {"Red": 1, "Green": 0, "Yellow": 1, "Blue": 1},
  {"Red": 0, "Green": 1, "Yellow": 0, "Blue": 1},
  {"Red": 1, "Green": 0, "Yellow": 1, "Blue": 1},  
  {"Red": 1, "Green": 1, "Yellow": 0, "Blue": 1},
  {"Red": 0, "Green": 1, "Yellow": 0, "Blue": 1},
  {"Red": 1, "Green": 0, "Yellow": 0, "Blue": 1},
]
print(pandas.DataFrame(ballots).sum())
print(f"{pandas.DataFrame(ballots).sum().idxmax()} wins")


Red       6
Green     4
Yellow    3
Blue      8
dtype: int64
Blue wins


In [91]:
# Instant Runoff Voting
import pandas

# Ranking a scored ballot because ranked ballots are bad, and annoying to compose.
ballots = [
  {"Red": 5, "Green": 0, "Yellow": 3, "Blue": 4},
  {"Red": 5, "Green": 0, "Yellow": 1, "Blue": 4},
  {"Red": 0, "Green": 5, "Yellow": 0, "Blue": 1},
  {"Red": 1, "Green": 2, "Yellow": 4, "Blue": 3}, 
  {"Red": 1, "Green": 0, "Yellow": 2, "Blue": 0},  
  {"Red": 1, "Green": 3, "Yellow": 0, "Blue": 1},
  {"Red": 0, "Green": 0, "Yellow": 5, "Blue": 0},
  {"Red": 5, "Green": 0, "Yellow": 0, "Blue": 4},
]

ballots = pandas.DataFrame(ballots)

def nextRound(ballots):
    # get highest rated candidate from each row, count the number of times it occurs
    round_count = pandas.DataFrame(ballots).apply(lambda x: x.idxmax(), axis=1).value_counts()
    print(round_count)
    # find the lowest count
    eliminated = round_count.idxmin()
    print(f"{eliminated =}")
    # remove the eliminated candidates from the ballots
    ballots.drop(eliminated, axis=1, inplace=True)
    return ballots

while len(ballots.columns) > 1:
    nextRound(ballots)
f'Winner: {ballots.columns[0]}'


Red       3
Yellow    3
Green     2
dtype: int64
eliminated ='Green'
Red       4
Yellow    3
Blue      1
dtype: int64
eliminated ='Blue'
Red       5
Yellow    3
dtype: int64
eliminated ='Yellow'


'Winner: Red'

In [92]:
# Single Transferrable Vote
import pandas

# Ranking a scored ballot because ranked ballots are bad, and annoying to compose.
ballots = [
  {"Red": 5, "Green": 0, "Yellow": 3, "Blue": 4},
  {"Red": 5, "Green": 0, "Yellow": 1, "Blue": 4},
  {"Red": 0, "Green": 5, "Yellow": 0, "Blue": 1},
  {"Red": 1, "Green": 2, "Yellow": 4, "Blue": 3}, 
  {"Red": 1, "Green": 0, "Yellow": 2, "Blue": 0},  
  {"Red": 1, "Green": 3, "Yellow": 0, "Blue": 1},
  {"Red": 0, "Green": 0, "Yellow": 5, "Blue": 0},
  {"Red": 5, "Green": 0, "Yellow": 0, "Blue": 4},
]

seats = 3
ballots = pandas.DataFrame(ballots)
quota = len(ballots) / (seats + 1) 
seated = []

# def nextRound(ballots):
#     # get highest rated candidate from each row, count the number of times it occurs
#     round_count = pandas.DataFrame(ballots).apply(lambda x: x.idxmax(), axis=1).value_counts()
#     print(round_count)
#     # find if any candidate has more than quota
#     print(quota, seated)
#     print(round_count[round_count > quota].index)
#     seated.append(round_count[round_count > quota].index)

#     # find the lowest count
#     eliminated = round_count.idxmin()
#     print(f"{eliminated =}")
#     # remove the eliminated candidates from the ballots
#     ballots.drop(eliminated, axis=1, inplace=True)
#     return ballots

# while len(ballots.columns) > 1:
#     nextRound(ballots)
# f'Winner: {ballots.columns[0]}'
